In [ ]:
dhl_shipments = [
    "5053537996",
    "6222779172",
    "4674719221",
    "4674780854",
    "8401066483",
    "8401020773",
    "8401039640",
    "4674673953",
    "8400951101",
    "9897583161",
    "8401098355",
    "1202505010",
    "2544861034",
    "2689870562",
    "2545040783",
    "2544939095",
    "2689898481",
    "2689921975",
    "2544920066"
]


In [49]:
import pandas as pd
from functions0_paths import API_details
api_key_DHL, api_secret_DHL = API_details()

In [3]:
tracking_number = "7137953071"

In [5]:
import requests

url = f"https://api-eu.dhl.com/track/shipments?trackingNumber={tracking_number}"

headers = {
    "DHL-API-Key": api_key_DHL,
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    # API request successful
    data = response.json()
    # Process the response data as needed
    data
else:
    # Handle errors
    print(f"Error {response.status_code}: {response.text}")


{'shipments': [{'id': '7137953071', 'service': 'express', 'origin': {'address': {'addressLocality': 'BARCELONA - SPAIN'}, 'servicePoint': {'url': 'http://www.dhl.es/en/country_profile.html', 'label': 'Origin Service Area'}}, 'destination': {'address': {'addressLocality': 'MANCHESTER - UK'}, 'servicePoint': {'url': 'http://www.dhl.co.uk/en/country_profile.html', 'label': 'Destination Service Area'}}, 'status': {'timestamp': '2023-10-11T11:16:00', 'location': {'address': {'addressLocality': 'MANCHESTER - UK'}}, 'statusCode': 'delivered', 'status': 'delivered', 'description': 'Delivered'}, 'details': {'proofOfDelivery': {'documentUrl': 'https://webpod.dhl.com/webPOD/DHLePODRequest?hwb=j5%2FfhrZetLbUlmk%2F8zOm2Q%3D%3D&pudate=usNTfeWtZbjutxhLillJ2Q%3D%3D&appuid=9WBxXOIFEL6MaYCOUzDZcQ%3D%3D&language=en&country=G0', 'signatureUrl': 'https://webpod.dhl.com/webPOD/DHLePODRequest?hwb=j5%2FfhrZetLbUlmk%2F8zOm2Q%3D%3D&pudate=usNTfeWtZbjutxhLillJ2Q%3D%3D&appuid=9WBxXOIFEL6MaYCOUzDZcQ%3D%3D&language

In [45]:
# Shipment number
shipment_num = data['shipments'][0]['id']
# Shipment Origin Date
shipm_origin_date = data['shipments'][0]['events'][-1]['timestamp']
# Shipment Origin Location
shipm_origin_location = data['shipments'][0]['events'][-1]['location']['address']['addressLocality']
# Number of pieces
num_pieces= data['shipments'][0]['details']['totalNumberOfPieces']
# Shipment destination
shipment_destination = data['shipments'][0]['destination']['address']['addressLocality']
# Carrier Status
carrier_status = data['shipments'][0]['status']['description']
# Last Update
last_update = data['shipments'][0]['events'][0]['timestamp']
# Last Location
last_location = data['shipments'][0]['events'][0]['location']['address']['addressLocality']
# Last Notification
last_notification = data['shipments'][0]['events'][0]['description']

'7137953071'

In [132]:
# Extract data from every shipment
all_results = []

for shipment in data['shipments']:
    # Shipment number
    shipment_num = data['shipments'][0]['id']
    # Shipment Origin Date
    shipm_origin_date = data['shipments'][0]['events'][-1]['timestamp']
    # Shipment Origin Location
    shipm_origin_location = data['shipments'][0]['events'][-1]['location']['address']['addressLocality']
    # Shipment destination
    shipment_destination = data['shipments'][0]['destination']['address']['addressLocality']
    # Number of pieces
    num_pieces= data['shipments'][0]['details']['totalNumberOfPieces']
    # Carrier Status
    carrier_status = data['shipments'][0]['status']['description']
    # Last Update
    last_update = data['shipments'][0]['events'][0]['timestamp']
    # Last Location
    last_location = data['shipments'][0]['events'][0]['location']['address']['addressLocality']
    # Last Notification
    last_notification = data['shipments'][0]['events'][0]['description']
    
    # Append extracted data to the results list
    all_results.append({
        "Client Reference": None,
        "Shipment Num.": shipment_num,
        "Origin Date": shipm_origin_date,
        "From": shipm_origin_location,
        "To": shipment_destination,
        "Num. of Pieces": num_pieces,
        "Carrier Status": carrier_status,
        "Last Update": last_update,
        "Last Location": last_location,
        "Last Action": last_notification,
        "Exception Notification": None
        })
    
# Create a DataFrame
dhl_df = pd.DataFrame(all_results)
dhl_df

,Client Reference,Shipment Num.,Origin Date,From,To,Num. of Pieces,Carrier Status,Last Update,Last Location,Last Action,Exception Notification
0,None,7137953071,2023-10-10T16:02:00,BARCELONA - SPAIN,MANCHESTER - UK,1,Delivered,2023-10-11T11:16:00,MANCHESTER - UK,Delivered,None


In [148]:
def clean_DHL(dhl_df):
    """
    Clean DHL DataFrame by formatting dates and calculating processing days.

    Parameters:
    - dhl_df (pd.DataFrame): Input DataFrame with columns 'Origin Date', 'Last Update', and 'Carrier Status'.

    Returns:
    - pd.DataFrame: Cleaned DataFrame with additional columns 'Hour', 'Processing Days'.
    - 'Origin Date' and 'Last Update' are formatted as 'dd-mm-yy'.
    - 'Processing Days' are calculated based on the difference between 'Last Update' and 'Origin Date'.
    - 'From', 'To', and 'Last Location' columns are transformed to title case.
    - Columns are rearranged for better readability.
    """

    # Import libraries inside the function
    import pandas as pd
    import numpy as np
    from datetime import date

    # Copy the DataFrame to avoid modifying the original
    df = dhl_df.copy()

    # Keep just dates for 'Origin Date' and 'Last Update', create last 'Hour'
    df['Origin Date'] = df['Origin Date'].astype(str).str.split('T').str[0]
    df['Hour'] = df['Last Update'].astype(str).str.split('T').str[1]
    df['Last Update'] = df['Last Update'].astype(str).str.split('T').str[0]
    
    # Convert 'Origin Date', 'Last Update' to datetime and then format as 'dd-mm-yy'
    df['Origin Date'] = pd.to_datetime(df['Origin Date'])#.dt.strftime('%d-%m-%y')
    df['Last Update'] = pd.to_datetime(df['Last Update'])#.dt.strftime('%d-%m-%y')
    
    # Create 'Processing Days'
    df['Processing Days'] = None
    
    for index, row in df.iterrows():
        if row['Carrier Status'] == "Delivered":
            # Calculate processing time for delivered shipments
            processing_days = pd.to_datetime(row['Last Update']) - pd.to_datetime(row['Origin Date'])
            df['Processing Days'] = processing_days
        else:
            current_date = date.today()
            # Convert to datetime.date
            current_date = pd.to_datetime(current_date, errors='coerce').date()  
            origin_date = pd.to_datetime(df['Origin Date']).dt.date
            df['Processing Days'] = current_date - origin_date
    
    # Convert 'Origin Date', 'Last Update' to datetime and then format as 'dd-mm-yy'
    df['Origin Date'] = pd.to_datetime(df['Origin Date']).dt.strftime('%d-%m-%y')
    df['Last Update'] = pd.to_datetime(df['Last Update']).dt.strftime('%d-%m-%y')
    
    # Applying title case transformation to column values 
    df['From'] = df['From'].str.title()
    df['To'] = df['To'].str.title()
    df['Last Location'] = df['To'].str.title()
    
    # Rearrange columns' order
    df = df[['Client Reference', 'Shipment Num.', 'Origin Date', 'From', 'To',
       'Num. of Pieces', 'Carrier Status', 'Last Update', 'Hour', 'Last Location',
       'Last Action', 'Processing Days', 'Exception Notification']]
    
    return df

# Set the maximum column width to display
pd.set_option('display.max_colwidth', None)
# Clean the DataFrame using the function
clean_dhl = clean_DHL(dhl_df)
clean_dhl

,Client Reference,Shipment Num.,Origin Date,From,To,Num. of Pieces,Carrier Status,Last Update,Hour,Last Location,Last Action,Processing Days,Exception Notification
0,None,7137953071,10-10-23,Barcelona - Spain,Manchester - Uk,1,Delivered,11-10-23,11:16:00,Manchester - Uk,Delivered,1 days,None


In [144]:
clean_dhl.columns

Index(['Client Reference', 'Shipment Num.', 'Origin Date', 'From', 'To',
       'Num. of Pieces', 'Carrier Status', 'Last Update', 'Last Location',
       'Last Action', 'Exception Notification', 'Hour', 'Processing Days'],
      dtype='object')

In [47]:
len(data['shipments'])

1

## Retrieved Data

In [149]:
#data

#### Access data for first shipment in "data"

In [19]:
# Shipment destination
data['shipments'][0]['destination']['address']['addressLocality']

'MANCHESTER - UK'

In [22]:
# Carrier Status
data['shipments'][0]['status']['description']

'Delivered'

In [32]:
# Last Update
data['shipments'][0]['events'][0]['timestamp']

'2023-10-11T11:16:00'

In [37]:
# Last Location
data['shipments'][0]['events'][0]['location']['address']['addressLocality']

'MANCHESTER - UK'

In [39]:
# Last Action / Notification
data['shipments'][0]['events'][0]['description']

'Delivered'

In [43]:
# Shipment Origin Date ([-1] ensures it is the first event registered)
data['shipments'][0]['events'][-1]['timestamp']

'2023-10-10T16:02:00'

In [44]:
# Shipment Origin
data['shipments'][0]['events'][-1]['location']['address']['addressLocality']

'BARCELONA - SPAIN'

In [29]:
# Number of pieces
data['shipments'][0]['details']['totalNumberOfPieces']

1